Create/update a `.env` file in the project root including the following environment variables.

AWS_DEFAULT_SAGEMAKER_BUCKET

AWS_PROFILE

In [1]:
LOCAL = False
START_PIPELINE = True

In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv("../../.env")

True

In [4]:
!mkdir -p ../../build
!rm -rf ../../build/*
!cp ../pipelines/recommendations_np/code/* ../../build/
!cp -r ../../src/pipeliner ../../build/

In [5]:
build_path = "../../build"

In [6]:
import os
AWS_DEFAULT_SAGEMAKER_BUCKET = os.environ.get("AWS_DEFAULT_SAGEMAKER_BUCKET", None)
DEFAULT_BUCKET_PREFIX = "pipelines"
if AWS_DEFAULT_SAGEMAKER_BUCKET is None:
    raise ValueError("AWS_DEFAULT_SAGEMAKER_BUCKET is not set")

Log into Docker registry with ECR credentials

In [7]:
!aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 141502667606.dkr.ecr.eu-west-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [8]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="pipeliner")["Role"]["Arn"]

role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/24/25 08:56:06] INFO     Found credentials from IAM Role:                                   ]8;id=821333;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=796826;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[04/24/25 08:56:09] INFO     Found credentials from IAM Role:                                   ]8;id=615405;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=706753;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

'arn:aws:iam::536135653944:role/ca-ml-sagemaker-notebook-production'

In [10]:
from sagemaker.workflow.pipeline_context import LocalPipelineSession, PipelineSession

if LOCAL:
    session = LocalPipelineSession(
        default_bucket=AWS_DEFAULT_SAGEMAKER_BUCKET,
        default_bucket_prefix=DEFAULT_BUCKET_PREFIX,
    )
    session.config = {"local": {"local_code": True}}
else:
    session = PipelineSession(
        default_bucket=AWS_DEFAULT_SAGEMAKER_BUCKET,
        default_bucket_prefix=DEFAULT_BUCKET_PREFIX,
    )

region = session.boto_region_name
default_bucket = session.default_bucket()

[04/24/25 08:56:10] INFO     Found credentials from IAM Role:                                   ]8;id=541249;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=810563;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [11]:
import pandas as pd
import numpy as np


ratings_data_path = "../pipelines/recommendations_np/data/user_item_interactions.csv.gz"

data_types = {"user_id": str, "item_id": str, "rating": np.float32}

user_item_interactions = pd.read_csv(
    ratings_data_path,
    compression="gzip",
    dtype=data_types,
    parse_dates=["date"],
)
user_item_interactions.head(3)

,user_id,item_id,date,interactions
0,U007714,I00372373,2024-09-08,1.0
1,U007714,I00605528,2024-09-08,1.0
2,U013522,I01182960,2024-09-08,2.0


In [12]:
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=ratings_data_path,
    desired_s3_uri=f"s3://{default_bucket}/{DEFAULT_BUCKET_PREFIX}/recommender/data",
)
input_data_uri

[04/24/25 08:56:12] INFO     Found credentials from IAM Role:                                   ]8;id=621373;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=481672;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

's3://cs-production-customer-analytics-sagemaker/pipelines/recommender/data/user_item_interactions.csv.gz'

In [13]:
import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import LocalPipelineSession
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig


class RecommenderPipeline:
    def create(
        self,
        role: str,
        name: str,
        session: sagemaker.Session,
        framework_version="1.2-1",
    ) -> Pipeline:
        self.local = isinstance(session, LocalPipelineSession)
        self.framework_version = framework_version

        instance_type = ParameterString(
            name="InstanceType",
            default_value="local" if self.local else "ml.m5.24xlarge",
        )

        input_data = ParameterString(
            name="user_item_interactions",
            default_value=input_data_uri,
        )

        image_uri = sagemaker.image_uris.retrieve(
            framework="sklearn",
            region=session.boto_region_name,
            version="1.2-1",
        )

        preprocessing_cache_config = CacheConfig(
            enable_caching=True,
            expire_after="p30d",  # 30 days
        )

        evaluation_cache_config = CacheConfig(
            enable_caching=True,
            expire_after="p30d",  # 30 days
        )

        training_cache_config = CacheConfig(
            enable_caching=True,
            expire_after="p30d",  # 30 days
        )

        processor = SKLearnProcessor(
            framework_version=framework_version,
            instance_type=instance_type,
            instance_count=1,
            base_job_name="sklearn-processor",
            role=role,
            sagemaker_session=session,
        )

        user_item_interactions_input = ProcessingInput(
            source=input_data,
            input_name="user_item_interactions",
            destination="/opt/ml/processing/input/data",
        )

        pipeliner_input = ProcessingInput(
            source=build_path + "/pipeliner",
            input_name="pipeliner",
            destination="/opt/ml/processing/input/code/pipeliner",
        )

        preprocessor_step = ProcessingStep(
            name="preprocessor",
            cache_config=preprocessing_cache_config,
            step_args=processor.run(
                inputs=[
                    user_item_interactions_input,
                    pipeliner_input,
                ],
                outputs=[
                    ProcessingOutput(
                        output_name="user_item_matrix",
                        source="/opt/ml/processing/output/user_item_matrix",
                    ),
                    ProcessingOutput(
                        output_name="item_similarity_matrix",
                        source="/opt/ml/processing/output/item_similarity_matrix",
                    ),
                    ProcessingOutput(
                        output_name="test_data",
                        source="/opt/ml/processing/output/test_data",
                    ),
                    ProcessingOutput(
                        output_name="user_encoder",
                        source="/opt/ml/processing/output/user_encoder",
                    ),
                    ProcessingOutput(
                        output_name="item_encoder",
                        source="/opt/ml/processing/output/item_encoder",
                    ),
                ],
                code=build_path + "/preprocessor.py",
            ),
        )

        sklearn_estimator = SKLearn(
            entry_point="item_based_recommender.py",
            source_dir=build_path,
            role=role,
            image_uri=image_uri,
            instance_type=instance_type,
            sagemaker_session=session,
            base_job_name="training_job",
            # hyperparameters=hyperparameters,
            enable_sagemaker_metrics=True,
            output_path=f"s3://{default_bucket}/{DEFAULT_BUCKET_PREFIX}/recommender/model"
        )

        item_training_step = TrainingStep(
            name="item_based_recommender",
            estimator=sklearn_estimator,
            cache_config=training_cache_config,
            inputs={
                "item_similarity_matrix": TrainingInput(
                    s3_data=preprocessor_step.properties.ProcessingOutputConfig.Outputs[
                        "item_similarity_matrix"
                    ].S3Output.S3Uri,
                    content_type="application/x-npz",
                ),
            },
        )

        evaluation_processor = SKLearnProcessor(
            framework_version=framework_version,
            instance_type=instance_type,
            instance_count=1,
            base_job_name="sklearn-evaluation",
            role=role,
            sagemaker_session=session,
        )

        evaluation_report = PropertyFile(
            name="item_based_evaluation",
            output_name="evaluation",
            path="evaluation.json",
        )

        evaluation_step = ProcessingStep(
            name="item_based_evaluation",
            cache_config=evaluation_cache_config,
            processor=evaluation_processor,
            inputs=[
                ProcessingInput(
                    source=item_training_step.properties.ModelArtifacts.S3ModelArtifacts,
                    destination="/opt/ml/processing/model",
                    input_name="model",
                ),
                ProcessingInput(
                    source=preprocessor_step.properties.ProcessingOutputConfig.Outputs[
                        "test_data"
                    ].S3Output.S3Uri,
                    destination="/opt/ml/processing/test_data",
                ),
                pipeliner_input,
            ],
            outputs=[
                ProcessingOutput(
                    output_name="evaluation",
                    source="/opt/ml/processing/evaluation",
                ),
            ],
            code="../pipelines/recommendations_np/code/item_based_evaluation.py",
            property_files=[evaluation_report],
        )

        return Pipeline(
            name=name,
            steps=[
                preprocessor_step,
                item_training_step,
                evaluation_step,
            ],
            sagemaker_session=session,
            parameters=[input_data, instance_type],
        )

In [14]:
pipeline = RecommenderPipeline().create(role=role, name="recommender", session=session)

[04/24/25 08:56:13] INFO     Defaulting to only available Python version: py3                     ]8;id=742684;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=519478;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=11823;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=752175;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

                    WARNING  The input argument instance_type of function                          ]8;id=860923;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=805724;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.retrieve) is a pipeline variable (<class                        
                             'sagemaker.workflow.parameters.ParameterString'>), which is                           
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     Defaulting to only available Python version: py3                     ]8;id=904852;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=383150;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


                    WARNING  instance_type is a PipelineVariable (<class                           ]8;id=356383;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/sklearn/estimator.py\estimator.py]8;;\:]8;id=124917;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/sklearn/estimator.py#294\294]8;;\
                             'sagemaker.workflow.parameters.ParameterString'>). Its interpreted                    
                             value in execution time should not be of GPU types since GPU training                 
                             is not supported for Scikit-Learn.                                                    

                    INFO     Found credentials from IAM Role:                                   ]8;id=229485;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=31441;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/steps.py:485: UserWarning: Profiling is enabled on the provided estimator. The default profiler rule includes a timestamp which will change each time the pipeline is upserted, causing cache misses. If profiling is not needed, set disable_profiler to True on the estimator.
  warnings.warn(msg)


                    WARNING  The input argument instance_type of function                          ]8;id=139239;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=348015;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.retrieve) is a pipeline variable (<class                        
                             'sagemaker.workflow.parameters.ParameterString'>), which is                           
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     Defaulting to only available Python version: py3                     ]8;id=397325;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=493124;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

In [15]:
import json

definition = json.loads(pipeline.definition())
[
    {"Name": step.get("Name"), "Type": step.get("Type")}
    for step in definition.get("Steps")
]

[04/24/25 08:56:14] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=228663;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=145397;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  The input argument instance_type of function                          ]8;id=442389;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=91190;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.get_training_image_uri) is a pipeline variable                  
                             (<class 'sagemaker.workflow.parameters.ParameterString'>), which is                   
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=853398;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=26970;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=858594;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=744453;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[{'Name': 'preprocessor', 'Type': 'Processing'},
 {'Name': 'item_based_recommender', 'Type': 'Training'},
 {'Name': 'item_based_evaluation', 'Type': 'Processing'}]

In [16]:
pipeline.upsert(role_arn=role)

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=196046;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=220594;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  The input argument instance_type of function                          ]8;id=222713;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=47200;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.get_training_image_uri) is a pipeline variable                  
                             (<class 'sagemaker.workflow.parameters.ParameterString'>), which is                   
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=738345;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=168225;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=870329;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=871935;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[04/24/25 08:56:15] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=650093;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=209858;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  The input argument instance_type of function                          ]8;id=158981;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=57739;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.get_training_image_uri) is a pipeline variable                  
                             (<class 'sagemaker.workflow.parameters.ParameterString'>), which is                   
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=729042;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=754356;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=618152;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=99556;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:536135653944:pipeline/recommender',
 'ResponseMetadata': {'RequestId': 'f407a442-02f5-464d-bb91-dfc68496eceb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f407a442-02f5-464d-bb91-dfc68496eceb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '79',
   'date': 'Thu, 24 Apr 2025 08:56:15 GMT'},
  'RetryAttempts': 0}}

In [17]:
execution = pipeline.start()

In [18]:
definition = json.loads(pipeline.definition())
steps = definition.get("Steps", {})
for step in steps:
    step_name = step.get("Name", None)
    step_type = step.get("Type", None)
    # step_args = step.get("Arguments", {})
    print(f"{step_name}: {step_type}")
    # print(step_args.keys())
    # output_config = step_args.get("ProcessingOutputConfig", step_args.get("OutputDataConfig", {}))
    # print(json.dumps(output_config, indent=4))

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=303351;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=936460;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  The input argument instance_type of function                          ]8;id=813331;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=756328;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.get_training_image_uri) is a pipeline variable                  
                             (<class 'sagemaker.workflow.parameters.ParameterString'>), which is                   
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=390981;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=364902;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=157927;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=607328;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

preprocessor: Processing
item_based_recommender: Training
item_based_evaluation: Processing


In [19]:
import time

execution_complete = False

while not execution_complete:
    current_execution = execution.describe()
    current_execution_name = current_execution.get("PipelineExecutionDisplayName", None)
    current_execution_status = current_execution.get("PipelineExecutionStatus", None)
    current_execution_failure_reason = current_execution.get("FailureReason", None)
    execution_complete = current_execution_status not in ('Executing','Stopping')
    print(f"\nstatus: {current_execution_status}")
    if current_execution_failure_reason:
        print(f"\nFailureReason: {current_execution_failure_reason}")
    if not execution_complete:
        time.sleep(60)

steps = execution.list_steps()
for step in steps[::-1]:
    step_name = step.get("StepName", None)
    step_status = step.get("StepStatus", None)
    failure_reasion = step.get("FailureReason", None)
    print(f"{step_name}: {step_status}")
    if failure_reasion is not None:
        print(f"    {failure_reasion}")
        
current_execution


status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Executing

status: Succeeded
preprocessor: Succeeded
item_based_recommender: Succeeded
item_based_evaluation: Succeeded


{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:536135653944:pipeline/recommender',
 'PipelineExecutionArn': 'arn:aws:sagemaker:eu-west-1:536135653944:pipeline/recommender/execution/t2dom68qjmzb',
 'PipelineExecutionDisplayName': 'execution-1745484975561',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'recommender',
  'TrialName': 't2dom68qjmzb'},
 'CreationTime': datetime.datetime(2025, 4, 24, 8, 56, 15, 480000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 4, 24, 9, 22, 53, 857000, tzinfo=tzlocal()),
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::536135653944:assumed-role/ca-ml-sagemaker-notebook-production/SageMaker',
   'PrincipalId': 'AROAXZVBM3I4LTE2HKMWE:SageMaker'}},
 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::536135653944:assumed-role/ca-ml-sagemaker-notebook-production/SageMaker',
   'PrincipalId': 'AROAXZVBM3I4LTE2HKMWE:SageMaker'}},
 'ResponseMetadata': {'RequestId': '40701147-6ea0-4262-963c-df5361c

In [47]:
sagemaker_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')

evaluation_step = steps[0]
evaluation_step_arn = evaluation_step['Metadata']['ProcessingJob']['Arn']
evaluation_step_arn

evaluation_jobs = sagemaker_client.list_processing_jobs(
    NameContains = 'item-based-evaluatio',
    SortOrder = 'Descending'
)

evaluation_job_name = evaluation_jobs['ProcessingJobSummaries'][0]['ProcessingJobName']

evaluation_job = sagemaker_client.describe_processing_job(
    ProcessingJobName=evaluation_job_name
)

s3_path = evaluation_job['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'] + '/evaluation.json'
s3_key = s3_path.split(f"s3://{AWS_DEFAULT_SAGEMAKER_BUCKET}/", 1)[1]

evaluation_results = s3_client.get_object(Bucket=AWS_DEFAULT_SAGEMAKER_BUCKET, Key=s3_key)
file_content = json.loads(evaluation_results["Body"].read().decode())
accuracy = file_content['binary_classification_metrics']['accuracy']['value']
print("%.6f" % accuracy)

0.000026
